In [2]:
import tensorflow as tf
import numpy as np
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data = loadmat("data.mat")

In [69]:
X_train = data["image"].astype('float32')
X_train = (X_train/128.0 - 1.0)
y_train = data["label"].astype('int32')
y_train = np.eye(4)[y_train].squeeze(axis = 1)

In [70]:
print("training data:",X_train.shape,", labels:",y_train.shape)

training data: (20780, 32, 32, 3) , labels: (20780, 4)


In [71]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [72]:
class input_data:
    def __init__(self, data_x, data_y, batch_size):
        self.data_x = data_x
        self.data_y = data_y
        self.batch_size = batch_size
        self.index = 0
        self.batch_x = []
        self.batch_y = []
        self.batch_number = int(self.data_x.shape[0] / self.batch_size)
        for i in range(self.batch_number):
            self.batch_x.append(self.data_x[i*self.batch_size:(i+1)*self.batch_size])
            self.batch_y.append(self.data_y[i*self.batch_size:(i+1)*self.batch_size])
            
    def next_batch(self):
        self.index = (self.index+1) % self.batch_number
        return self.batch_x[self.index], self.batch_y[self.index]

In [73]:
train_batch = input_data(X_train, y_train, 64)

In [74]:
x, y = train_batch.next_batch()

In [75]:
def init_weight(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1))

In [76]:
def init_bias(shape):
    return tf.Variable(tf.constant(0.1, shape=shape))

In [77]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, [1,1,1,1], padding="SAME")

In [78]:
def maxpooling(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [79]:
def conv_layer(input_x, filter_shape):
    W = init_weight(shape=filter_shape)
    b = init_bias(shape=[filter_shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

In [80]:
def fc_layer(input_x, hidden_nodes):
    layer_shape = [int(input_x.get_shape()[1]),hidden_nodes]
    W = init_weight(layer_shape)
    b = init_bias([hidden_nodes])
    return tf.matmul(input_x, W) + b

In [81]:
tf.reset_default_graph()
input_x = tf.placeholder(dtype=tf.float32, shape=[None,32,32,3])
input_y = tf.placeholder(dtype=tf.int32, shape=[None,4])

In [86]:
conv10 = conv_layer(input_x, [3,3,3,32])
conv11 = conv_layer(conv10, [3,3,32,32])
max_pool1 = maxpooling(conv11)
conv20 = conv_layer(max_pool1, [3,3,32,32])
conv20 = conv_layer(max_pool1, [3,3,32,32])
max_pool2 = maxpooling(conv20)

conv2_flat = tf.reshape(max_pool2, [-1,32//4 * 32//4 * 32])
fc1 = tf.nn.relu(fc_layer(conv2_flat, 128))

hold_prob = tf.placeholder(tf.float32)
fc_dropout = tf.nn.dropout(fc1,keep_prob=hold_prob)

pred_y = fc_layer(fc_dropout, 4)

In [93]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=input_y, logits=pred_y))
tf_pred = tf.nn.softmax(pred_y)

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

tf_acc = tf.reduce_mean(tf.to_float(tf.equal(tf.argmax(tf_pred, 1), tf.argmax(input_y, 1))))

init = tf.global_variables_initializer()

In [94]:
batch_x, batch_y = train_batch.next_batch()
batch_x.shape

(64, 32, 32, 3)

In [95]:

steps = 1000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = train_batch.next_batch()

        sess.run(optimizer,feed_dict={input_x:batch_x,input_y:batch_y,hold_prob:0.5})
        
        #PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            print('Currently on step {}'.format(i))
            acc = sess.run(tf_acc,feed_dict={input_x:X_val,input_y:y_val,hold_prob:1.0})
            print(acc)
            
    print('Finish!')


Currently on step 0
0.2454283
Currently on step 100
0.62993264
Currently on step 200
0.7078922
Currently on step 300
0.7475938
Currently on step 400
0.76997113
Currently on step 500
0.77333975
Currently on step 600
0.7836862
Currently on step 700
0.80365735
Currently on step 800
0.7846487
Currently on step 900
0.79956686
Finish!
